In [1]:
!pip install yfinance --upgrade --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.18
    Uninstalling yfinance-0.2.18:
      Successfully uninstalled yfinance-0.2.18


In [2]:
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
end_data = datetime.now().strftime('%Y-%m-%d')
start_data = datetime.now() - timedelta(days=365*3)

In [4]:
data_renner = yf.download("LREN3.SA", start=start_data, end=end_data, progress=False)
data_soma = yf.download("SOMA3.SA", start=start_data, end=end_data, progress=False)

In [5]:
stocks = ["LREN3.SA", "SOMA3.SA"]
#p_data = pd.DataFrame()
p_data=[]

#for s in stocks:
#    p_data[s] = yf.download(s, start=start_data, end=end_data, progress=False)['Adj Close']

for s in stocks:
    data = yf.download(s, start=start_data, end=end_data, progress=False)
    p_data.append(data)

df = pd.concat(p_data, keys=stocks, names=['stocks','Date'])
df.head()

Open       High        Low      Close  Adj Close  \
stocks   Date                                                                
LREN3.SA 2020-06-22  43.483265  44.028912  42.054657  42.600304  40.187969   
         2020-06-23  42.768959  43.056667  41.270905  41.538769  39.186550   
         2020-06-24  41.062569  41.895920  40.001030  40.725258  38.478081   
         2020-06-25  40.685574  41.588375  40.139923  40.487156  38.253117   
         2020-06-26  40.447472  40.665730  39.485146  40.278816  38.056271   

                      Volume  
stocks   Date                 
LREN3.SA 2020-06-22  5344580  
         2020-06-23  9054025  
         2020-06-24  6441961  
         2020-06-25  7003100  
         2020-06-26  9233041

In [6]:
df = df.reset_index()
df=df[df['Date'] >= '2022-01-01']
df.head()

,stocks,Date,Open,High,Low,Close,Adj Close,Volume
379,LREN3.SA,2022-01-03,24.450001,24.650000,23.080000,23.080000,22.204279,11665200
380,LREN3.SA,2022-01-04,23.250000,23.549999,22.440001,22.799999,21.934900,16175500
381,LREN3.SA,2022-01-05,22.719999,22.980000,21.650000,21.690001,20.867018,13078300
382,LREN3.SA,2022-01-06,21.870001,22.940001,21.440001,22.799999,21.934900,21405100
383,LREN3.SA,2022-01-07,22.750000,23.389999,22.270000,22.990000,22.117691,15523500


In [7]:
fig = px.line(df, x='Date',
              y='Close',
              color='stocks',
              title="Stock Market Performance for the Last 2 Years")
fig.show()

In [8]:
fig = px.area(df, x='Date', y='Close', color='stocks',
              facet_col='stocks',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Renner and Soma')
fig.show()

In [9]:
df['MA9'] = df.groupby('stocks')['Close'].rolling(window=9).mean().reset_index(0, drop=True)
df['MA21'] = df.groupby('stocks')['Close'].rolling(window=21).mean().reset_index(0, drop=True)

for stocks, group in df.groupby('stocks'):
    print(f'Moving Averages for {stocks}')
    print(group[['MA9', 'MA21']])

Moving Averages for LREN3.SA
           MA9       MA21
379        NaN        NaN
380        NaN        NaN
381        NaN        NaN
382        NaN        NaN
383        NaN        NaN
..         ...        ...
739  21.102222  19.790953
740  21.328889  19.988572
741  21.385555  20.183333
742  21.370000  20.264762
743  21.398889  20.356667

[365 rows x 2 columns]
Moving Averages for SOMA3.SA
            MA9       MA21
1094        NaN        NaN
1095        NaN        NaN
1096        NaN        NaN
1097        NaN        NaN
1098        NaN        NaN
...         ...        ...
1454  10.893333  10.372857
1455  11.008889  10.443333
1456  11.114444  10.537619
1457  11.235556  10.630476
1458  11.362222  10.714286

[365 rows x 2 columns]


In [10]:
for stocks, group in df.groupby('stocks'):
    fig = px.line(group, x='Date', y=['Close', 'MA9', 'MA21'],
                  title=f"{stocks} Moving Averages")
    fig.show()

In [11]:
# create a DataFrame with the stock prices of Apple and Microsoft
renner = df.loc[df['stocks'] == 'LREN3.SA', ['Date', 'Close']].rename(columns={'Close': 'LREN3.SA'})
soma = df.loc[df['stocks'] == 'SOMA3.SA', ['Date', 'Close']].rename(columns={'Close': 'SOMA3.SA'})
df_corr = pd.merge(renner, soma, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='LREN3.SA', y='SOMA3.SA',
                 trendline='ols',
                 title='Correlation between Renner and SOMA',
                 width=1200, height=800)

fig.show()